By Amy Weng

Word2Vec Continuous Skip-Gram Model Word Embedding and Visualization

https://radimrehurek.com/gensim/models/word2vec.html

In [123]:
import pandas as pd
from words import remove_stopwords
from gensim.scripts import word2vec2tensor
from gensim.models import Word2Vec
from gensim.models import Phrases

def embed(df,title):
    data = df.text.values.tolist()
    # preprocess and remove stopwords
    data = remove_stopwords(data)
    # Train a bigram detector.
    bigram_transformer = Phrases(data)
    # Train a skip-gram model with the bigram detector 
    model = Word2Vec(bigram_transformer[data], min_count=5,sg=1)
    # save model so we can reload later  
    model.save('/home/rapiduser/Materials/embeddings/'+title+'.model')
    # save model in a format that can be converted to tensor TSV 
    model.wv.save_word2vec_format('/home/rapiduser/Materials/embeddings/tensor/'+title+'.model')
    return(model)

def similar(model,word,num):
    if word in model.wv.key_to_index.keys():
        words = []
        # first = 0
        # score = 0
        for w, s in model.wv.most_similar(word,topn=num):
            # if first == 0:
            #     score = s
            #     first += 1
            words.append(w)
            # words.append((w,s))
        # print(word + ': '+' '.join(words))
        # print('The most similar n-gram has cosine distance '+str(score))
        # print('\n')
        return ' '.join(words)

def comparePair(model,word1,word2):
    if word1 and word2 in model.wv.key_to_index.keys():
        return model.wv.similarity(word1, word2)
    return None
        # print('Cosine similarity between ' + word1 + ' and ' + word2 + ': ',model.wv.similarity(word1, word2))

def tensor(f_name):
    word2vec2tensor.word2vec2tensor(
        '/home/rapiduser/Materials/embeddings/tensor/'+f_name+'.model',
        '/home/rapiduser/ECBC-21-22/Text_Files/Embeddings TSV/'+f_name,
        binary=False)

In [2]:
myCSV = '/home/rapiduser/Materials/topic model/publica/eic_monopoly.csv'

readFile = pd.read_csv(myCSV)

# read text information into a dataframe
publica_eic_monopoly = embed(readFile,'publica_eic_monopoly')
tensor('publica_eic_monopoly')

In [126]:
consumption = 'consumption|consume|consuming|consumed|consumer|consumers|conumption|conume|conuming|conumed|conumer|conumers'
publica = 'abroad|act|administrators|admiral|adventurers|africa|african|aid|ally|ambaador|ambassador|amboyna|amsterdam|argier|army|austerity|authority|bank|bankrupt|bankrupts|banks|bantam|bavaria|bill|bills|body politic|bond|britain|bullion|bullionist|calico|calicoes|captain|cargo|cargoes|cent|cents|charles|charters|china|chocolate|christiandom|cinnamon|citizens|civil war|civilian|clothier|clothiers|cloths|coffee|coin|coined|commerce|commercial|commiioner|commissioner|commissioners|commodities|commodity|commonwealth|companies|company|constitution|consumables|consume|consumer|consuming|consumption|controversies|controversy|conume|conumption|copper|corn|corporate|corporation|council|councils|country|court|courten|courts|credit|creditor|creditors|crisis|crisis|currency|debt|debtor|decoctor|decree|decrees|defendant|dutch|duties|east india company|east indies|east-india company|east-india-company|eat-india company|economic|economy|emperor|empire|employment|enemy|england|england|estate|europe|european|exchange|exchequer|exportation|exported|extortion|factories|factory|fih|fihing|fiscal|fish|fishery|fishing|foreign|free trade|germany|ginger|gold|goods|government|governor|governors|grant|grants|gunpowder|herring|herrings|hillings|hollanders|holy roman empire|house of commons|house of lords|importation|imported|income|indenture|india|india|indian|indies|indonesia|industry|interest|interest rate|interest rates|interet|interloper|interlopers|ireland|james|joint-stock|king|kingdom|kingdom|kings|land|law|laws|license|licenser|licensing|london|lord|low countries|majesties|majesty|majeties|manufacture|manufacturers|manufactures|manufacturing|market|markets|mary|massacre|mercantile|merchandise|merchant|merchants|military|mines|mint|minted|mogul|money|moneys|monies|monopolie|monopolies|monopolion|monopolist|monopolium|monopolization|monopolize|monopolizer|monopolizes|monopoly|monopolion|monopolye|monopolyes|monoply|monopolise|monopolising|monopolists|monopolizers|monopolised|monoopolies|monopolits|monopolers|monopoliing|mughal|nation|national|naval|navigation|navy|negroes|netherlands|netherlands|nets|oppression|parliament|parliament|pay|pence|pences|pepper|petitioner|petitioners|petre|phaulkon|pirate|pirates|plaintiff|plantations|planters|pleads|politic|political|pound|pounds|power|price|prices|prince|princes|privileges|proclamation|proecutor|proecutors|profitable|prohibition|property|proprietors|protection|provincial|public|publicke|queen|quote|restoration|revenue|revenues|revolution|ruler|salt|saltpetre|scandal|scandalous|scotland|seamen|shilling|shillings|shipping|ships|siam|siam|silk|silver|solicitors|spain|spanish|spending|spices|spices|sterling|stock|strength|stuart|subject|substance|supply|syam|taxation|taxes|tea|textiles|tobacco|tories|tory|trade|tradesmen|trading|traffic|traffique|treasure|treaties|treatise|treatises|treaty|truces|tunis|turkish|unprofitable|usurer|usury|war|whig|whigs|william|wood|woollen|work'
religio = 'abnegatio|abraham|absolution|abundance|adam|adultery|almighty|altar|anabaptists|anathema|annihilate|anoint|anointing|antichrist|apocalypse|apostle|archangel|armageddon|arrogant|ascension|atone|atonement|avarice|baptism|bathsheba|bible|biblical|bishop|bishop|bishops|blasphemy|bless|blessed|blessing|born-again|bread|brother|caesar|calvinist|candles|canon|catholic|cessans|chalice|chapel|chaplain|charity|cheat|cheating|cherub|christ|christian|christianity|chrit|church|cleansing|clergy|coming|commandments|communion|condemn|condemnation|confess|confession|congregation|conscience|consecration|contempt|contrite|contrition|conviction|covenant|create|creation|creed|cross|crown|cuckold|curse|damask|damn|damnation|damned|david|day|deacon|death|deceit|deceitful|deceive|deceive|defile|defiled|delilah|demon|depravity|desire|desires|destruction|deuteronomy|devil|disciple|disciples|disobey|divine|doctrine|ecumenical|endure|entice|enticed|epistle|eternal|evangelical|evangelicalism|eve|evil|excommunication|exile|exodus|faith|false|falsehood|fellowship|forgive|forgiveness|fortune|fraud|freedom|fundamental|generation|genesis|gentile|gluttony|god|godly|gospel|grace|greed|green|grievance|guilt|guilty|hallelujah|heaven|hebrew|hebrews|hell|heresy|heresy|holiness|holy|human|hypocrisy|incest|indulgence|indulgent|integrity|isaiah|james|jehovah|jesus|job|judaism|judgment|justification|justify|knowledge|lechery|leviticus|lie|lies|liturgy|lord|lucifer|lucrum|luke|lust|lustful|lustfully|luxury|manifestation|mark|mass|matthew|mendicancy|messiah|minister|ministry|mission|missionary|modesty|moral|moses|obedience|obey|offering|offerings|ordained|ordinance|ordinate|ordination|orthodox|pagan|paganism|pagans|palace|papacy|papist|parable|parables|parish|parishioner|passover|pastor|penance|perfect|persecution|peter|pleasant|pope|popery|pray|prayer|predestination|prejudicial|presbyterian|presbyterians|pride|priesthood|prodigal|prodigality|prohibition|prophecy|prophet|protestant|proverb|proverbs|providence|psalm|psalms|purgatory|radiance|rapture|reap|reconcile|reconciliation|redeem|redeemed|redemption|reform|reformed|reincarnate|reincarnation|reject|rejected|rejoice|rejoiced|religion|render|repent|resurrect|resurrection|revelation|rich|righteous|righteousness|ritual|roman|rome|root|sabbath|sacrament|sacred|sacrifice|sacrifices|saint|salvation|samaritan|samson|samuel|sanctified|sanctify|sanctuary|satan|satanic|satanism|save|saved|saved|savior|scripture|scriptures|second|secure|security|self-denial|sermon|simony|sin|sinful|sins|sister|slave|slaves|sloth|soul|sown|spirit|splendor|splendor|station|stations|sumptuary|sun|supper|tempt|ten|tenant|tenants|testament|theft|thrive|timothy|tongues|tower|transgression|tribulation|trinity|trust|truth|twelve|unfaithful|union|universal|unnatural|vanity|velvet|venial|virgin|vision|vulgate|wealth|whore|wicked|wickedness|wine|wise|witness|word|words|worship|zion'
medica = 'apothecaries|apothecary|asthma|asthma|atrophy|bad|bezoar|bile|bleed|bleeding|blood|body|brain|breathing|cancer|canker|circulate|circulation|clots|clotting|cold|constitution|consume|consumption|contamination|corpora|corpus|corrupt|corruptio|corruption|corruptionem|cough|cure|cured|dead|death|decay|decay|deceased|defect|degenerate|degeneration|destroy|diabetes|diagnosis|disease|dissolution|distemper|drinks|dry|dying|enfeeblement|envy|evacuation|exercise|faculties|fatal|fatality|fever|fiber|fog|healthy|heart|hemorrhage|hepatitis|herb|herbs|hot|hysteria|illness|impairment|inanition|insane|jealousy|languish|leech|leeches|liver|lung|malaria|medical|medicine|medicines|melancholy|miasma|mind|moist|morbid|mortal|mortality|mouth|nerves|nervous|opiate|opium|patient|patients|perversion|perverto|phlegm|phyician|phyicians|physician|physicians|plague|plague|pox|prognosis|putrid|putrid|remedies|remedy|sana|sane|sanguine|sick|sickness|smells|smoke|sores|spirit|spoil|spoiling|stomach|supple|surgeon|surgeons|swelling|tetrid|therapeutic|tuberculosis|ulcer|unhealthy|unwholesome|upset|vein|vein|vessels|vice|vita|vital|vitiare|vitiation|vitium|vomit|waste|wasting|wholesome|wintergreen|zodiac'

In [121]:
import os 
import re 
df = pd.DataFrame(columns=('year','similar','publica ratio','religio ratio','medica ratio'))
for year in range(1660,1700+1):
    similarWords = ''
    if (str(year)+'_eic.model') in os.listdir('/home/rapiduser/Materials/embeddings'):
        model = Word2Vec.load('/home/rapiduser/Materials/embeddings/' + str(year)+'_eic.model')
        for term in consumption.split('|'):
            if similar(model,term,50) != None:
                similarWords += similar(model,term,100)
        if similarWords != '':
            total = len(similarWords.split(' '))
            p = len(re.findall(publica, similarWords))/total
            r = len(re.findall(religio, similarWords))/total
            m = len(re.findall(medica, similarWords))/total
            data = {'year':year,'similar':similarWords,
                'publica ratio':p,'religio ratio':r,
                'medica ratio':m}
            df = df.append(data,ignore_index=True)
outfile = '/home/rapiduser/ECBC-21-22/Text_Files/cosine similarities/eic_consumption_ratios.csv'
df.to_csv(outfile)

In [127]:
import os 
import re 
df = pd.DataFrame(columns=('year','similar','publica ratio','religio ratio','medica ratio'))
for year in range(1660,1700+1):
    similarWords = ''
    pCosine,rCosine,mCosine,pNum,rNum,mNum,pRatio,rRatio,mRatio = 0,0,0,0,0,0,0,0,0
    if (str(year)+'_eic.model') in os.listdir('/home/rapiduser/Materials/embeddings'):
        model = Word2Vec.load('/home/rapiduser/Materials/embeddings/' + str(year)+'_eic.model')
        for word1 in consumption.split('|'):
            if word1 in model.wv.key_to_index.keys():
                for p in publica.split('|'):
                    if p in model.wv.key_to_index.keys():
                        pCosine += comparePair(model,word1,p)
                        pNum += 1 
                for r in religio.split('|'):
                    if r in model.wv.key_to_index.keys():
                        rCosine += comparePair(model,word1,r)
                        rNum += 1 
                for m in medica.split('|'):
                    if m in model.wv.key_to_index.keys():
                        mCosine += comparePair(model,word1,m)
                        mNum += 1 

        if pNum != 0: 
            pRatio = pCosine/pNum
        if rNum != 0: 
            rRatio = rCosine/rNum
        if mNum != 0: 
            mRatio = mCosine/mNum
        
        data = {'year':year,'similar':similarWords,
            'publica ratio':pRatio,'religio ratio':rRatio,
            'medica ratio':mRatio}
        df = df.append(data,ignore_index=True)
outfile = '/home/rapiduser/ECBC-21-22/Text_Files/cosine similarities/eic_consumption_ratios2.csv'
df.to_csv(outfile)

In [128]:
print(df)

    year similar publica ratio religio ratio medica ratio
0   1660                   142           103           24
1   1661                     0             0            0
2   1662                     0             0            0
3   1663                     0             0            0
4   1664                  1330          1414          364
5   1665                     0             0            0
6   1666                   158           102           36
7   1667                     0             0            0
8   1668                     0             0            0
9   1669                     0             0            0
10  1670                     0             0            0
11  1672                   203           204           59
12  1673                     0             0            0
13  1674                     0             0            0
14  1675                     0             0            0
15  1676                   855           780          220
16  1677      

In [12]:
dir = '/home/rapiduser/Materials/Texts/eic_subsets'
for year in range(1696,1700+1):
    readFile = pd.read_csv(dir+'/'+str(year)+'.csv')
    embed(readFile,str(year)+'_eic')

In [27]:
from words import monopoly
m = monopoly.split('|')
for w in m:
    similar(publica_eic_monopoly,w,10)

monopolies: laws_land freedom_trade retrained restrain restrained retrain exclude_others wrong legally privilege
The most similar n-gram has cosine distance 0.9599364995956421
monopoly: exclusive depending regulated_company joint_stocks monopolies retrained privilege wrong majeties_subjects allege
The most similar n-gram has cosine distance 0.9274605512619019
monopolise: allegations advantages_profits ingroed naturally_follow befalls free_encumbrances trengthen fact_allege corporation_excluding default_paying
The most similar n-gram has cosine distance 0.9918798804283142
monopolising: enriched greater_prices remote_parts riches_strength rates_exchange feel_effects encreaing increase_navigation increase_seamen attempting
The most similar n-gram has cosine distance 0.9856666326522827
monopolizers: lil vat_extent butchers served_apprentice tillage stil betowed consuming afford_gains trademan
The most similar n-gram has cosine distance 0.991474449634552
monopolised: olely forces_forts diff

In [ ]:
terms1 = 'body_politic|body_politick|public_utility|public_affairs|wicked|illegal|engrossing|odious|evil|repugnant|arbitrary|popery|papist|bribery|remedies|remedy|monopoly_evils|circulation|disease|wasting|waste'
terms2 = 'monopoly|monopolies|monopolise|monopolising|monopolizers|monopolised|corruption|corrupt|corrupting'
list1 = terms1.split('|')
list2 = terms2.split('|')
for word1 in list1: 
    for word2 in list2: 
        comparePair(publica_eic_monopoly,word1,word2)
    print('\n')

In [31]:
similar(publica_eic_monopoly,'maladies',25)
similar(publica_eic_monopoly,'malady',25)

maladies: remedies_disease maxims_government need_proof stroke prevails harms survive gain_impunity dispens reasonableness bless illustration hyde vacation geometry leading decumbiture teaching humanly_speaking curses frightful captivity racks deliberating taken_pains
The most similar n-gram has cosine distance 0.991988480091095
malady: knowledge_ans facility anguish anger sureties_good ill_consequences speculation entitles faintly release impertinent unbias modety provoke profes smart parting_money sincere composition pity creator wherefore_leave kin disposition policies_contracts
The most similar n-gram has cosine distance 0.9886937141418457


In [ ]:
consumption = 'consumption|consume|consuming|consumed|conume|conumption|conuming|conumed'
c = consumption.split('|')
for w in c:
    similar(publica_eic_monopoly,w,10)

In [ ]:
corruption = 'corruption|corrupt|corrupted|corruptions|corrupting'
corrupt = corruption.split('|')
for w in corrupt:
    similar(publica_eic_monopoly,w,10)